In [5]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import os, re
import time

# url = 'https://www.kaigokensaku.mhlw.go.jp/23/index.php?action_kouhyou_pref_search_search=true&method=search&p_count=50&p_offset=0'
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36'}

# 従業員テーブルをリサーチする
def find_jyugyouin_df(df_list):
    for df in df_list:
        if '総従業者数' in df.to_string():
            return df

# 抽出データ格納用フォルダーを作成する
if not os.path.exists('data'):
    os.mkdir('data')

# 使用時に指定が必要なパラメータ
KEN = '13' # この数字を変更すれば、県違いでリサーチ可能
PAGES = 0 # ページ数を指定ください。0の場合、デフォルトは全ページとなる、各ページに50項目有り

# 総ページ数を取得する
url_page = 'https://www.kaigokensaku.mhlw.go.jp/{}/index.php?action_kouhyou_pref_search_search=true&method=search&p_count=50&p_offset=0'.format(KEN)
url_page_info = requests.get(url_page, headers=header)
total = json.loads(url_page_info.text)['pager']['total']
pages_ = int(total / 50) + 1
if PAGES == 0:
    pages = pages_
else:
    pages = PAGES

# 全体URLのリストを生成する
urls = ['https://www.kaigokensaku.mhlw.go.jp/{}/index.php?action_kouhyou_pref_search_search=true&method=search&p_count=50&p_offset={}'.format(KEN,i) for i in range(0,pages*50,50)] # pages*50
for i,url in enumerate(urls):
    infos = []
    print('スクレイピング中:{}/{}'.format(i+1,pages))
    webInfo = requests.get(url, headers=header)
    jsonInfo = json.loads(webInfo.text)
    data = jsonInfo['data']
    for item in data:
        print('   {}...'.format(item.get('JigyosyoName')))
        tmp = {
            '施設名': item.get('JigyosyoName').replace('　',''),
            '住所':item.get('JigyosyoJyusho'),
            'TEL':item.get('JigyosyoTel'),
            '施設コード_1':item.get('JigyosyoCd'),
            '施設コード_2':item.get('JigyosyoSubCd'),
            'サービスコード':item.get('ServiceCd'),
            'バージョンコード':item.get('VersionCd'),
            '郵便番号':item.get('JigyosyoYubinbangou').replace(',','-'),
            'FAX':item.get('JigyosyoFax',)
        }
        
        time.sleep(1)

        # 法人名称,介護サービス種類を取得する
        houjin_url = 'https://www.kaigokensaku.mhlw.go.jp/{}/index.php?action_kouhyou_detail_004_kihon=true&JigyosyoCd={}-{}&ServiceCd={}'.format(
            KEN, tmp['施設コード_1'], tmp['施設コード_2'], tmp['サービスコード'])
        houjinInfo = requests.get(houjin_url, headers=header)
        houjinbs = BeautifulSoup(houjinInfo.text, 'lxml')
        n = houjinbs.select('.houjin_number_check')
        try:
            tmp['法人'] = n[1].text
        except:
            tmp['法人'] = "情報なし"
        try:
            v = houjinbs.select('#serviceName')[0].text
            tmp['介護サービスの種類'] = v
        except:
            tmp['介護サービスの種類'] = "情報なし"


            # 従業者情報を取得する
        jyugyouin_url = 'https://www.kaigokensaku.mhlw.go.jp/{}/index.php?action_kouhyou_detail_004_kani=true&JigyosyoCd={}-{}&ServiceCd={}'.format(
            KEN, tmp['施設コード_1'], tmp['施設コード_2'], tmp['サービスコード'])
        jyugyouinInfo = requests.get(jyugyouin_url, headers =header)
        if not os.path.exists('data/{}'.format(tmp['施設名'])):
            try:
              os.mkdir(f"data/{tmp['施設名']}")
            except FileNotFoundError:
              print("FileNotFoundError")
        df_table_jyugyouin = pd.read_html(jyugyouinInfo.text)
        df_jyugyouin = find_jyugyouin_df(df_table_jyugyouin)
        try:
          df_jyugyouin.to_csv('data/{}/従業者.csv'.format(tmp['施設名']), encoding='cp932')
        except FileNotFoundError:
              print("FileNotFoundError")

        # 運営タブにあるすべてのテーブルを取得する
        unei_url = 'https://www.kaigokensaku.mhlw.go.jp/{}/index.php?action_kouhyou_detail_004_unei=true&JigyosyoCd={}-{}&ServiceCd={}'.format(
            KEN, tmp['施設コード_1'], tmp['施設コード_2'], tmp['サービスコード'])
        uneiInfo = requests.get(unei_url, headers=header)
        temp_unei = uneiInfo.text
        pattern = re.compile(r"<tr class='isEtcRow'>.*?</tr>",re.S)
        temp_unei = re.sub(pattern,'',temp_unei)
        temp_unei = re.sub(r'<img width="32" height="32" alt="-" .*?>', '-', temp_unei)
        temp_unei = re.sub(r'<img width="32" height="32" alt="○" .*?>', '○', temp_unei)
        df_list = pd.read_html(temp_unei)
        labels = [
            '1．利用者の権利擁護のための取組',
            '2．利用者本位の介護サービスの提供',
            '3．相談、苦情等の対応のために講じている措置',
            '4．サービスの内容の評価や改善等',
            '5．サービスの質の確保、透明性の確保等のための外部機関等との連携',
            '6．適切な事業運営の確保',
            '7．事業所の運営管理、業務分担、情報の共有等',
            '8．安全管理及び衛生管理',
            '9．情報の管理、個人情報保護等',
            '10．その他、介護サービスの質の確保のために行っていること'
        ]
        for i,item in enumerate(df_list):
            if i==0:
                continue
            item.drop(item.columns[1:3], axis=1, inplace=True)
            try:
              item.to_csv('data/{}/{}.csv'.format(tmp['施設名'],labels[i-1]), encoding='cp932')
            except FileNotFoundError:
                print("FileNotFoundError")

        infos.append(tmp)
    df = pd.DataFrame(infos)
    if i!= 0:
        df.to_csv('info.csv', mode='a', encoding='cp932', errors="ignore",header=None)
    else:
        df.to_csv('info.csv',mode='a', encoding='cp932',errors="ignore")
print('スクレイピングは完了しました！')

スクレイピング中:1/325
   (ひゅーまんらいふけあなかむらばしのやど）...
   (医）色空会　お茶の水整形外科...
   (株)アンティ・ケア...
   (株)ベスト・ケアー　ヘルパーステーション　練馬...
   (株)国立メディカルヘルパーステーションひろ...
   15デイサービス「グリーンデイ」用賀...
   24時間ホームケアえごた...
   2丁目の広場デイサービス...
   AIENケアサービス...
   AIリハ 南久が原Ⅰ...
   AIリハ尾山台...
   ASHCARE 訪問看護ステーション...
   Act...
   BPリハビリStudio...
   BPリハビリ訪問看護ステーション...
   CARE・DEPOTエンゼル...
   Care-Story居宅介護支援事業所...
   Care‐Story...
   Carrent東京...
   CoCo訪問リハビリ・看護ステーション...
   DAYS BLG !...
   DHCスマイルガーデン上板橋店...
   DHCスマイルガーデン浮間舟渡店...
   DHCスマイルガーデン赤塚店...
   DHC小規模多機能型居宅介護事業所「赤塚店」...
   DHC居宅介護支援事業所「浮間舟渡店」...
   EC・アシスト...
   Each Care世田谷...
   Each　Care　世田谷...
   FCM訪問看護ステーションすみだ...
   FRESH WALK...
   FiveStar訪問看護・栄養管理Station...
   FreeLife...
   FreeLife...
   GENKINEXT 小金井前原町...
   GENKINEXT　練馬旭町...
   GENKINEXT　足立竹ノ塚...
   GENKINEXT御徒町...
   GENKINEXT板橋蓮根...
   GENKINEXT秋津...
   GENKINEXT葛飾小菅...
   GENKINEXT調布多摩川...
   GENKI　NEXT　八王子明神町...
   GENKI　NEXT　練馬豊玉北...
   GENKI　NEXT東十条...
   GIVER訪問介護ステーション...
   GIVER訪問看護ステーション.

KeyboardInterrupt: 

In [ ]:
import shutil
shutil.make_archive('data', 'zip', root_dir='/content/data')

In [ ]:
import shutil
shutil.rmtree('/content/data')